In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import re,math
import pandas as pd

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
pop = pd.read_csv('../data/user_log.csv',sep='|')
pop.head()

,uid,action,jobNo,invoice,dateTime,source,url,deviceType
0,1245540518330,viewJob,8338295.0,43125162,2016-07-20 16:59:36,app,NaN,1.0
1,1245540518330,viewJob,8338295.0,43125162,2016-07-08 01:04:27,app,NaN,1.0
2,1245540518330,applyJob,8338295.0,43125162,NaN,app,NaN,0.0
3,1245540518330,viewJob,5117491.0,22662550,2016-07-20 12:26:29,app,NaN,1.0
4,1245540518330,applyJob,5117491.0,22662550,NaN,app,NaN,0.0


In [3]:
#算ACTION的次數_先去除不要的欄位
pop = pop.drop(['invoice','source','url','deviceType'], axis = 1)


,uid,jobNo,action,count
0,2,208337.0,saveJob,1
1,2,208337.0,viewJob,2
2,2,896465.0,viewJob,1
3,2,1463885.0,viewJob,1
4,2,3553643.0,viewJob,1


In [ ]:
#算ACTION的次數
group = pop[pop.action != 'viewCust'].groupby(['uid','jobNo','action']).size().reset_index(name='count')
group.head()

In [5]:
#先調整jobNo格式
group['jobNo'] = group['jobNo'].apply(np.int64)
group['uid'] = group['uid'].apply(np.str)
group['jobNo'] = group['jobNo'].apply(np.str)

In [6]:
#算ACTION的次數_轉置
group_convert = group.set_index([ 'uid' , 'jobNo' , 'action' ] ) 

In [7]:
group_covt = group_convert.unstack(level= 2)  
group_covt[:10]

count                
action      applyJob saveJob viewJob
uid jobNo                           
10  4518869      NaN     NaN     2.0
    4716652      NaN     NaN     5.0
    6160395      NaN     NaN     1.0
    6335818      NaN     NaN     1.0
    6338116      NaN     1.0     NaN
    6518964      NaN     2.0     1.0
    6906951      NaN     NaN     1.0
    6969102      NaN     NaN     1.0
    7227871      NaN     NaN     3.0
    7297838      NaN     NaN     1.0

In [8]:
turn_uid = group_covt.reset_index('uid')
turn_jobNo = turn_uid.reset_index('jobNo')
turn_jobNo[:10]

jobNo uid    count                
action              applyJob saveJob viewJob
0       4518869  10      NaN     NaN     2.0
1       4716652  10      NaN     NaN     5.0
2       6160395  10      NaN     NaN     1.0
3       6335818  10      NaN     NaN     1.0
4       6338116  10      NaN     1.0     NaN
5       6518964  10      NaN     2.0     1.0
6       6906951  10      NaN     NaN     1.0
7       6969102  10      NaN     NaN     1.0
8       7227871  10      NaN     NaN     3.0
9       7297838  10      NaN     NaN     1.0

## Test

In [28]:
job_category = pd.read_csv('mapping_job_class.csv')
job_category.head()

,invoice,jobno,jobcat1,big,mid,manager_ind
0,4541302,788127,2008001006,研發相關類,工程研發類,0.0
1,22099218,881206,2008001016,研發相關類,工程研發類,0.0
2,84149738,962383,2008001009,研發相關類,工程研發類,0.0
3,76027628,3338171,2003001009,財會／金融專業類,財務／會計／稅務類,0.0
4,4541302,3716258,2010001009,操作／技術／維修類,操作／技術類,0.0


In [29]:
#取其中一類：資訊軟體系統類
#去除不需要的欄位
big = job_category[job_category['big'] == '資訊軟體系統類']
big = big.drop(['invoice','jobcat1','mid','big','manager_ind'], axis = 1)
big = big.rename(columns = {'jobno' : 'jobNo'})
big['jobNo'] = big['jobNo'].apply(np.str)

In [30]:
#預設是inner join
join = pd.merge(final, big, how ='inner')
join.head()

,applyJob,jobNo,saveJob,uid,viewJob
0,0.0,4143534,0.0,1001,1.0
1,0.0,4143534,0.0,1005022347373,2.0
2,0.0,4143534,0.0,1005022347435,1.0
3,0.0,4143534,0.0,1005022347518,9.0
4,0.0,4143534,0.0,1005022347676,1.0


In [44]:
#此產業類別：共有多少 distinct的 uid & job
n_users = join.uid.unique().shape[0]
n_items = join.jobNo.unique().shape[0]
str(n_users) , str(n_items)

('261611', '6191')

In [45]:
#此產業類別：總筆數
len(join)

2217054

In [46]:
#此產業類別：no apply（key值為uid + jobNo）
no_apply = join[join['applyJob'] == 0]
no_apply.head()

,applyJob,jobNo,saveJob,uid,viewJob
0,0.0,4143534,0.0,1001,1.0
1,0.0,4143534,0.0,1005022347373,2.0
2,0.0,4143534,0.0,1005022347435,1.0
3,0.0,4143534,0.0,1005022347518,9.0
4,0.0,4143534,0.0,1005022347676,1.0


In [47]:
# 此產業類別：no apply筆數
len(no_apply)

1971418

In [53]:
# 此產業類別：有apply （key值為uid + jobNo）
apply1 = join[join['applyJob'] != 0]
apply1.head()

,applyJob,jobNo,saveJob,uid,viewJob
25,1.0,4143534,1.0,1013612284552,9.0
30,1.0,4143534,0.0,1022202216812,8.0
36,1.0,4143534,0.0,1022202221078,0.0
42,11.0,4143534,3.0,103079215198,15.0
44,1.0,4143534,0.0,1030792152411,3.0


In [54]:
#把有APPLY的人由次數型轉變為註記型
apply2 = apply1.assign(applyJob = apply1['applyJob'] / apply1['applyJob'])
apply2.head()

,applyJob,jobNo,saveJob,uid,viewJob
25,1.0,4143534,1.0,1013612284552,9.0
30,1.0,4143534,0.0,1022202216812,8.0
36,1.0,4143534,0.0,1022202221078,0.0
42,1.0,4143534,3.0,103079215198,15.0
44,1.0,4143534,0.0,1030792152411,3.0
